Install Required Libraries

In [1]:
!pip install geopandas
!pip install geemap
!pip install earthengine-api

Authenticates Earth Engine

In [2]:
import ee
import geemap

# Authenticate with Earth Engine
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize(project='minaseyedi-internship')

Adjust File Input

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Availible Bansa in Sentinel2

In [4]:
info = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").first().getInfo()
for i in range(len(info['bands'])):
  print(info['bands'][i]['id'])

B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
QA10
QA20
QA60
MSK_CLASSI_OPAQUE
MSK_CLASSI_CIRRUS
MSK_CLASSI_SNOW_ICE


Modify the SentinelImageDownloader Class

In [5]:
import ee
import geemap
import geopandas as gpd

class SentinelImageDownloader:
  def __init__(self, shapefile_path, start_date, end_date, output_folder):
    self.shapefile_path = shapefile_path
    self.start_date = start_date
    self.end_date = end_date
    self.output_folder = output_folder
    self.aoi = self.load_shapefile()
    self.cloud_masking = self.maskS2clouds

  def load_shapefile(self):
    # Load the shapefile using GeoPandas
    gdf = gpd.read_file(self.shapefile_path)

    # Reproject the GeoDataFrame to WGS84 (EPSG:4326) for compatibility with Earth Engine
    gdf = gdf.to_crs(epsg=4326)

    # Combine all geometries using union_all instead of unary_union
    geometry = gdf.geometry.union_all()

    # Get the coordinates of the extior boundary
    if geometry.geom_type == 'GeomtryColllection':
      coords = [list(geom.extrior.coords) if geom.geom_type == 'Polygon' else [] for geom in geometry]
      coords = [coord for sublist in coords for coord in sublist]
    else:
      coords = list(geometry.exterior.coords)

    return ee.Geometry.Polygon(coords)

  def maskS2clouds(self, image):
    # Select the QA60 band (cloud and cirrus bitmask)
    qa = image.select('QA60')

    # Define bitmasks for clouds and cirrus
    cloudBitMask = 1 << 10  # Cloud bit
    cirrusBitMask = 1 << 11  # Cirrus bit

    # Apply bitwise AND to isolate cloud-free pixels
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
      qa.bitwiseAnd(cirrusBitMask).eq(0)
    )

    # Update the image mask and return the masked image
    return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])



  def get_sentinel1_images(self):
    sentinel1_ImageCollections = ee.ImageCollection("COPERNICUS/S1_GRD") \
      .filterBounds(self.aoi) \
      .filterDate(self.start_date, self.end_date) \
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
      .select(['VV', 'VH'])   # Select 'VV' and 'VH' bands
    count = sentinel1_ImageCollections.size().getInfo()
    print(f"Number of Sentinel-1 images: {count}")

    return sentinel1_ImageCollections


  def get_sentinel2_images(self):
    sentinel2_ImageCollections = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
      .filterBounds(self.aoi) \
      .filterDate(self.start_date, self.end_date) \
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
      .map(self.maskS2clouds) \
      .select(['B4', 'B3', 'B2'])

    count = sentinel2_ImageCollections.size().getInfo()
    print(f"Number of Sentinel-2 images: {count}")
    return sentinel2_ImageCollections


  def get_ESA_WorldCover_v100_image(self):
    count = ee.ImageCollection('ESA/WorldCover/v100').size().getInfo()
    print(f"Number of Sentinel-100 images: {count}")

    return ee.ImageCollection('ESA/WorldCover/v100').first()


  def get_ESA_WorldCover_v200_image(self):
    count = ee.ImageCollection('ESA/WorldCover/v200').size().getInfo()
    print(f"Number of Sentinel-200 images: {count}")

    return ee.ImageCollection('ESA/WorldCover/v200').first()


  def download_images(self, image_collection, prefix):
    def download_image_to_drive(image):
      # Export the image as a GeoTIFF to Drive
      image_id = image.get('system:index').getInfo()
      filename = f"{output_folder}{prefix}_{image_id}"
      print(filename)

      task = ee.batch.Export.image.toDrive(**{
          'image': image.clip(self.aoi),
          'description': f"export_{prefix}_{image_id}",
          'fileNamePrefix': f"{prefix}_{image_id}",
          'folder': 'GEE_Downloads',
          'scale': 10,
          'region': self.aoi,
          'crs': 'EPSG:4326'
      })
      task.start()

    # Check if the input is an ImageCollection or a single Image
    if isinstance(image_collection, ee.ImageCollection):
      image_list = image_collection.toList(image_collection.size())  # Get the list of images
      for i in range(image_list.size().getInfo()):  # Iterate through the list
        image = ee.Image(image_list.get(i))
        download_image_to_drive(image)  # Download the image
    elif isinstance(image_collection, ee.Image):
      # If it's a single image, download it directly
      download_image_to_drive(image_collection)

  def run(self):
    sentinel1_image_collection = self.get_sentinel1_images()
    sentinel2_image_collection = self.get_sentinel2_images()
    esa_100_image_collection = self.get_ESA_WorldCover_v100_image()
    esa_200_image_collection = self.get_ESA_WorldCover_v200_image()

    print("Downloading Sentinel-1 images...")
    if sentinel1_image_collection.size().getInfo() > 0:
      self.download_images(sentinel1_image_collection, "S1")
    else:
      print("No Sentinel-1 images found.")

    print("Downloading Sentinel-2 images...")
    if sentinel2_image_collection.size().getInfo() > 0:
      self.download_images(sentinel2_image_collection, "S2")
    else:
      print("No Sentinel-2 images found.")

    print("Downloading ESA v100 images...")
    if esa_100_image_collection:
      self.download_images(esa_100_image_collection, "ESA100")
    else:
      print("No ESA v100 images found.")

    print("Downloading ESA v200 images...")
    if esa_200_image_collection:
      self.download_images(esa_200_image_collection, "ESA200")
    else:
      print("No ESA v200 images found.")


if __name__ == "__main__":
  # Define your parameters
  shapefile_path = '/content/drive/MyDrive/GEE_Exe/ArcGIS/Region.shp'  # Update the path accordingly
  start_date = "2021-01-01"
  end_date = "2021-02-01"
  output_folder = '/content/drive/MyDrive/GEE_Exe/GEE_Downloads/'  # Save to your Google Drive folder

  # Create an instance of the downloader class
  downloader = SentinelImageDownloader(shapefile_path, start_date, end_date, output_folder)

  # Run the image download process
  downloader.run()


Number of Sentinel-1 images: 6
Number of Sentinel-2 images: 1
Number of Sentinel-100 images: 1
Number of Sentinel-200 images: 1
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/S1_S1A_IW_GRDH_1SDV_20210103T143603_20210103T143628_035975_0436EC_13EF
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/S1_S1A_IW_GRDH_1SDV_20210104T023711_20210104T023736_035982_04371E_B7A3
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/S1_S1A_IW_GRDH_1SDV_20210115T143602_20210115T143627_036150_043D10_903D
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/S1_S1A_IW_GRDH_1SDV_20210116T023710_20210116T023735_036157_043D43_672F
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/S1_S1A_IW_GRDH_1SDV_20210127T143602_20210127T143627_036325_044326_D380
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/S1_S1A_IW_GRDH_1SDV_20210128T023710_20210128T023735_036332_04435F_CAA7
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/S2_20210112T072251_20210112T072250_T39SWV
/content/drive/MyDrive/GEE_Exe/GEE_Downloads/ESA100_2020
/content/drive/MyDrive/GEE_Exe/G